In [ ]:
import yfinance as yf
import pandas as pd

# Download NVDA stock data at 5-minute intervals for the past 30 days
df = yf.download(tickers='NVDA', period='60d', interval='5m')

# Flatten columns if multi-indexed
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

# Convert index (Datetime) from UTC to US/Eastern
df.index = df.index.tz_convert('US/Eastern')

# Preview the DataFrame
print(df.head())

# Optional: Save to CSV
df.to_csv('nvda_60days_5min.csv')

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg,
    NavigationToolbar2Tk,
)
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mdates

class CandlestickApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Candlestick Chart Viewer")

        self.load_btn = tk.Button(root, text="Load Stock CSV", command=self.load_csv)
        self.load_btn.pack(pady=10)

        self.canvas = None
        self.toolbar = None

    def load_csv(self):
        file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
        if not file_path:
            return

        try:
            # Load CSV and parse datetime
            df = pd.read_csv(file_path, parse_dates=['Datetime'])

            # Remove timezone info
            df['Datetime'] = df['Datetime'].dt.tz_localize(None)

            # Convert to matplotlib date format
            df['Date_Num'] = mdates.date2num(df['Datetime'])

            # Reorder columns
            ohlc = df[['Date_Num', 'Open', 'High', 'Low', 'Close']]

            self.plot_candlestick(ohlc)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to load or parse CSV:\n{str(e)}")

    def plot_candlestick(self, ohlc_data):
        # Clean up previous widgets
        if self.canvas:
            self.canvas.get_tk_widget().destroy()
        if self.toolbar:
            self.toolbar.destroy()

        fig, ax = plt.subplots(figsize=(12, 6))
        candlestick_ohlc(ax, ohlc_data.values, width=0.0005, colorup='green', colordown='red')

        ax.xaxis_date()
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M\n%Y-%m-%d'))
        ax.set_title("Candlestick Chart")
        ax.set_xlabel("Time")
        ax.set_ylabel("Price")
        plt.xticks(rotation=45)
        fig.tight_layout()

        self.canvas = FigureCanvasTkAgg(fig, master=self.root)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

        # Add zoom/pan toolbar
        self.toolbar = NavigationToolbar2Tk(self.canvas, self.root)
        self.toolbar.update()
        self.toolbar.pack(side=tk.TOP, fill=tk.X)

# Run it
if __name__ == "__main__":
    root = tk.Tk()
    app = CandlestickApp(root)
    root.mainloop()